#  JHU CSSE  - Novel Coronavirus (COVID-19) Cases dataset

In [ ]:
import altair as alt
from functools import reduce
import numpy as np
import os
import pandas as pd
import wget

In [ ]:
countries = ['US', 'Italy', 'China', 'Spain', 'Germany', 'France', 'Iran', 'United Kingdom', 'Switzerland']

In [ ]:
BASE_PATH = os.path.abspath(os.path.curdir)

### Dataset

**Novel Coronavirus (COVID-19) Cases, provided by JHU CSSE**

The Johns Hopkins CSSE aggregates data from primary sources, such as the World Health Organisation, national, and regional public health institutions. This data is made freely available and is updated every day

[Source](https://github.com/CSSEGISandData/COVID-19)

## Getting data

In [ ]:
# url of the raw csv dataset
# urls = [
#     'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
#     'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
#     'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
# ]
# [wget.download(url) for url in urls]

In [ ]:
def get_files(base_path):

    for d in os.listdir(base_path):
        if os.path.isfile(os.path.join(base_path, d)):
            if len(d) == 1:
                yield os.path.join(base_path, d)
        else:
            for f in get_files(os.path.join(base_path, d)):
                yield f


urls = list(get_files(BASE_PATH))
urls

In [ ]:
# confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
# deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
# recovered_df = pd.read_csv('time_series_covid19_recovered_global.csv')

In [ ]:
df_list = [pd.read_csv(file) for file in urls]

In [ ]:
# confirmed_df.columns
df_list[0].columns

In [ ]:
# confirmed_df.columns[4:]
df_list[0].columns[4:]

## Unpivot DataFrames from current wide format into long format

In [ ]:
# dates = confirmed_df.columns[4:]

# confirmed_df_long = confirmed_df.melt(
#     id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
#     value_vars=dates, 
#     var_name='Date', 
#     value_name='Confirmed'
# )
# deaths_df_long = deaths_df.melt(
#     id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
#     value_vars=dates, 
#     var_name='Date', 
#     value_name='Deaths'
# )
# recovered_df_long = recovered_df.melt(
#     id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
#     value_vars=dates, 
#     var_name='Date', 
#     value_name='Recovered'
# )

In [ ]:
dates = df_list[0].columns[4:]

confirmed_df_long = df_list[0].melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates,
    var_name='Date', 
    value_name='Confirmed'
)
deaths_df_long = df_list[1].melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
recovered_df_long = df_list[2].melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)

We have to remove recovered data for Canada due to mismatch issue (🤷‍♂ Canada recovered data is counted by Country-wise rather than Province/State-wise)

In [ ]:
recovered_df_long = recovered_df_long[recovered_df_long['Country/Region']!='Canada']

Now we use merge() to merge the 3 DataFrames one after another

In [ ]:
# Merging confirmed_df_long and deaths_df_long
full_table = confirmed_df_long.merge(
  right=deaths_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)
# Merging full_table and recovered_df_long
full_table = full_table.merge(
  right=recovered_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

In [ ]:
full_table.head()

## Data Cleaning

In [ ]:
full_table.dtypes

In [ ]:
full_table['Date'] = pd.to_datetime(full_table['Date'])

In [ ]:
full_table.isna().sum()

In [ ]:
full_table['Recovered'] = full_table['Recovered'].fillna(0)

In [ ]:
ship_rows = (full_table['Province/State'].str.contains('Grand Princess') | 
                        full_table['Province/State'].str.contains('Diamond Princess') | 
                        full_table['Country/Region'].str.contains('Diamond Princess') | 
                        full_table['Country/Region'].str.contains('MS Zaandam'))
full_ship = full_table[ship_rows]

In [ ]:
full_table = full_table[~(ship_rows)]

## Data Aggregation

In [ ]:
# Active Case = confirmed - deaths - recovered
full_table.loc[:, 'Active'] = full_table.loc[:, 'Confirmed'] - full_table.loc[:, 'Deaths'] - full_table.loc[:, 'Recovered']

In [ ]:
full_table.head()

In [ ]:
group_by = ['Date', 'Country/Region']
full_grouped = full_table.groupby(group_by)[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index()
full_grouped.tail()

In [ ]:
# new cases 
temp = full_grouped.groupby(['Country/Region', 'Date', ])[['Confirmed', 'Deaths', 'Recovered']]
# sums columns and calculate the difference row wise
temp = temp.sum().diff().reset_index()
# set as nan the 1st record of each Country
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan
# renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']
# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])
# filling na with 0
full_grouped = full_grouped.fillna(0)
# fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')
# 
full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

In [ ]:
full_grouped.tail()

In [ ]:
temp['Country/Region'].shift(1)

## Store results

In [ ]:
# full_grouped.to_csv('COVID-19-time-series-clean-complete.csv')

## Data Exploration

In [ ]:
# full_grouped = pd.read_csv('COVID-19-time-series-clean-complete.csv', parse_dates=['Date'])
grouped = full_grouped[full_grouped['Country/Region'].isin(countries)]

In [ ]:
base = alt.Chart(grouped).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

In [ ]:
red = alt.value('#f54242')
base.encode(y='Confirmed').properties(title='Total Confirmed') | base.encode(y='Deaths', color=red).properties(title='Total Deaths')

In [ ]:
red = alt.value('#f54242')
base.encode(y='New cases').properties(title='Daily new cases') | base.encode(y='New deaths', color=red).properties(title='Daily new deaths')

## Showing the Coronavirus spread for certain countries

In [ ]:
selected_countries = full_grouped[full_grouped['Country/Region'].isin(countries)]

In [ ]:
interval = alt.selection_interval()
circle = alt.Chart(selected_countries).mark_circle().encode(
    x='monthdate(Date):O',
    y='Country/Region',
    color=alt.condition(interval, 'Country/Region', alt.value('lightgray')),
    size=alt.Size('New cases:Q',
        scale=alt.Scale(range=[0, 3000]),
        legend=alt.Legend(title='Daily new cases')
    ) 
).properties(
    width=1000,
    height=300,
    selection=interval
)
bars = alt.Chart(selected_countries).mark_bar().encode(
    y='Country/Region',
    color='Country/Region',
    x='sum(New cases):Q'
).properties(
    width=1000
).transform_filter(
    interval
)

In [ ]:
circle & bars